In [39]:
driver = webdriver.Chrome(service=ser, options=op)
driver.maximize_window()
driver.get(page_link_to_open)
sleep(1)
homeCard = driver.find_elements(By.CSS_SELECTOR , "a.slider-item")
[c.get_attribute('href') for c in homeCard]

[]

In [10]:
import pandas as pd

In [2]:
from selenium import webdriver  # install selenium
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from time import sleep

In [3]:
path_to_browser = "./chromedriver.exe"
ser = Service(path_to_browser)
op = webdriver.ChromeOptions()
op.add_argument('headless')
page_link_to_open = "https://www.redfin.com/zipcode/00601"

In [5]:
# https://www.redfin.com/CA/San-Francisco/401-Harrison-St-94105/unit-46B/home/144064341
page_link_to_open2 = 'https://www.redfin.com/CA/San-Francisco/401-Harrison-St-94105/unit-46B/home/144064341'

driver = webdriver.Chrome(service=ser, options=op)
driver.maximize_window()
driver.get(page_link_to_open2)
sleep(0.2)
data = driver.find_elements(By.CSS_SELECTOR , ".statsValue")
[price, beds, bath, area] = [c.text for c in data]
stress_address = driver.find_elements(By.CSS_SELECTOR , ".street-address")[0].text
city_and_state = driver.find_elements(By.CSS_SELECTOR , ".dp-subtext")[0].text

# home_facts = driver.find_elements(By.CSS_SELECTOR, ".keyDetailsList")[0] \
#                    .find_elements(By.CSS_SELECTOR, "div.keyDetail")
# home_fact_index = [fact.find_elements(By.CSS_SELECTOR , "span")[0].text for fact in home_facts]
# home_fact_text = [fact.find_elements(By.CSS_SELECTOR, "span.text-right")[0].text for fact in home_facts]
# year_built = home_fact_text[home_fact_index.index('Year Built')] if 'Year Built' in home_fact_index else None
# lot_size = home_fact_text[home_fact_index.index('Lot Size')] if 'Lot Size' in home_fact_index else None

basic_infor = driver.find_elements(By.ID, "basicInfo")[0]
basic_infor_label = [e.text for e in basic_infor.find_elements(By.CSS_SELECTOR, "span.table-label")]
basic_infor_value = [e.text for e in basic_infor.find_elements(By.CSS_SELECTOR, "div.table-value")]
year_built = basic_infor_value[basic_infor_label.index('Year Built')] if 'Year Built' in basic_infor_label else None
county = basic_infor_value[basic_infor_label.index('County')] if 'County' in basic_infor_label else None
lot_size = basic_infor_value[basic_infor_label.index('Lot Size')] if 'Lot Size' in basic_infor_label else None

scores = driver.find_elements(By.CSS_SELECTOR , ".walk-score")
if scores:
    scores = scores[0].find_elements(By.CSS_SELECTOR , ".score")
    trademark = [score.find_elements(By.CSS_SELECTOR , ".walkscore-trademark")[0].text[:-1] for score in scores]
    score_value = [score.find_elements(By.CSS_SELECTOR , "div.percentage > span.value")[0].text for score in scores]
    walk_score = score_value[trademark.index('Walk Score')] if 'Walk Score' in trademark else None
    transit_score = score_value[trademark.index('Transit Score')] if 'Transit Score' in trademark else None
    bike_score = score_value[trademark.index('Bike Score')] if 'Bike Score' in trademark else None


In [6]:
price, beds, bath, area, stress_address[:-1], city_and_state, year_built, lot_size, county, walk_score, transit_score, bike_score

('$7,900,000',
 '3',
 '3',
 '3,214',
 '401 Harrison St Unit 46B',
 'San Francisco, CA 94105',
 '2014',
 '—',
 'San Francisco County',
 '93',
 '100',
 '57')

In [86]:
trademark

['Walk Score', 'Transit Score', 'Bike Score']

In [87]:
walk_score = score_value[trademark.index('Walk Score')] if 'Walk Score' in trademark else None
transit_score = score_value[trademark.index('Transit Score')] if 'Transit Score' in trademark else None
bike_score = score_value[trademark.index('Bike Score')] if 'Bike Score' in trademark else None

In [13]:
all_href = pd.read_csv('./data/property_href.csv')
all_href

,zipcode,href
0,zipcode,href
1,01001,https://www.redfin.com/MA/Agawam/24-Rowley-St...
2,01001,https://www.redfin.com/MA/Agawam/418-Meadow-S...
3,01001,https://www.redfin.com/MA/Agawam/611-Suffield...
4,01001,https://www.redfin.com/MA/Agawam/418-Meadow-S...
...,...,...
104815,15475,https://www.redfin.com/PA/Brownsville/110-Hop...
104816,15475,https://www.redfin.com/PA/Grindstone/418-Shag...
104817,15475,https://www.redfin.com/PA/Grindstone/210-Marb...
104818,15477,https://www.redfin.com/PA/Roscoe/417-Main-St-...


In [15]:
all_href['href']

KeyError: 'href'